In [1]:
with open('doc.txt',"r",encoding="utf-8") as f:
    raw_data=f.read()

In [2]:
import tiktoken
tokenizer=tiktoken.get_encoding("gpt2")

In [3]:
enc_text=tokenizer.encode(raw_data)
print(len(enc_text))

1570


In [4]:
context_length=4
x=enc_text[:context_length]
y=enc_text[context_length]
print(f"{x}--->{y}")

[8582, 234, 247, 383]--->6869


In [5]:
for i in range (1,context_length+1):
    context=enc_text[:i]
    desired=enc_text[i]
    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))
   



� ---> �
� ---> �
🌙 --->  The
🌙 The --->  Moon


In [6]:
import torch
from torch.utils.data import Dataset

class GPTdatasetv1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunks = token_ids[i : i + max_length]
            target_chunks = token_ids[i + 1 : i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunks, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunks, dtype=torch.long))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
 

In [7]:
from torch.utils.data import DataLoader
import tiktoken

def create_dataloader_v1(
        txt,
        batch_size=4,
        max_length=256,
        stride=128,
        shuffle=True,
        drop_last=True,
        num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTdatasetv1(txt, tokenizer, max_length, stride)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

In [8]:
import torch
dataloader=create_dataloader_v1(raw_data,batch_size=8,max_length=4,stride=1,shuffle=False)
data_iter=iter (dataloader)
inputs,target=next(data_iter)
print("INPUTS\n",inputs)
print("targets\n",target)



INPUTS
 tensor([[8582,  234,  247,  383],
        [ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851]])
targets
 tensor([[ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851],
        [ 276, 6785,  851,  317]])


>token embeddings


In [9]:
import torch
vocab_size=50257
output_dim=256
torch.manual_seed(123)  #random value alwase be constant when it is run 
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035,  ...,  1.3337,  0.0771, -0.0522],
        [ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [-1.3152, -0.0677, -0.1350,  ..., -0.3181, -1.3936,  0.5226],
        ...,
        [ 0.5871, -0.0572, -1.1628,  ..., -0.6887, -0.7364,  0.4479],
        [ 0.4438,  0.7411,  1.1263,  ...,  1.2091,  0.6781,  0.3331],
        [-0.2537,  0.1446,  0.7203,  ..., -0.2134,  0.2144,  0.3006]],
       requires_grad=True)


In [10]:
input_ids=torch.tensor([1,4,5,4,3,5])
print(embedding_layer(input_ids)) 


tensor([[ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [ 0.5229,  0.1553,  0.5247,  ..., -0.2004,  0.8093, -0.6667],
        [-0.3162,  1.2700, -0.0903,  ..., -0.4098,  0.4978, -0.3721],
        [ 0.5229,  0.1553,  0.5247,  ..., -0.2004,  0.8093, -0.6667],
        [ 0.2579,  0.3420, -0.8168,  ..., -0.4840, -0.2713, -0.0774],
        [-0.3162,  1.2700, -0.0903,  ..., -0.4098,  0.4978, -0.3721]],
       grad_fn=<EmbeddingBackward0>)


In [11]:
max_length=4
dataloader=create_dataloader_v1(raw_data,batch_size=8,max_length=max_length,stride=1,shuffle=False)
data_iter=iter (dataloader)
inputs,target=next(data_iter)



In [12]:
print("INPUTS\n",inputs)
print("shape\n",inputs.shape)

INPUTS
 tensor([[8582,  234,  247,  383],
        [ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851]])
shape
 torch.Size([8, 4])


In [13]:
token_embeddings=embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [14]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)
pos_embeddings=pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


In [15]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


# IMPLEMENTING SELF ATTENTION WITH TRAINABLE WEIGHTS


In [16]:
import torch
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)

In [17]:
inputs.shape


torch.Size([6, 3])

In [18]:
x_2 = inputs[1] #A
d_in = inputs.shape[1] #B
d_out = 2 #C

In [19]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out),
requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out),
requires_grad=False)

In [20]:
print(W_query)

print(W_key)

print(W_value)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]])
Parameter containing:
tensor([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]])


In [21]:
query_2=x_2@W_query
key_2=x_2@W_key
value_2=x_2@W_value
print(query_2)

tensor([0.4306, 1.4551])


>for all input 


In [22]:
keys = inputs @ W_key
values = inputs @ W_value
queries = inputs @ W_query
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)
print("queries.shape:", queries.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])
queries.shape: torch.Size([6, 2])


In [23]:
attn_scores = queries @ keys.T # omega
print(attn_scores)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [24]:
d_k = keys.shape[-1]
attn_weights = torch.softmax(attn_scores/ d_k**0.5, dim=-1)
print(attn_weights)
print(d_k)

tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])
2


In [25]:

context_vec = attn_scores @ values
print(context_vec)

tensor([[1.8346, 4.9351],
        [2.5091, 6.7530],
        [2.4766, 6.6657],
        [1.3772, 3.7065],
        [1.1948, 3.2184],
        [1.7832, 4.7978]])


# lets wrap the entire attention mechanism in a class 

In [32]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

	•	super() returns a special object that represents the parent class.
	•	Then super().__init__() calls the parent class’s __init__.
	•	This avoids writing the parent class name manually 


In [34]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
#GPT-2's original implementation doesn't use bias for Q/K/V
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

In [36]:
#likely used 789 to get different initial weights than sa_v1 (which used seed 123), so the outputs would differ.

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# masked attention mechanism

In [43]:
context_length = attn_scores.shape[0]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.9231,   -inf,   -inf,   -inf,   -inf,   -inf],
        [1.2705, 1.8524,   -inf,   -inf,   -inf,   -inf],
        [1.2544, 1.8284, 1.7877,   -inf,   -inf,   -inf],
        [0.6973, 1.0167, 0.9941, 0.5925,   -inf,   -inf],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707,   -inf],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


In [55]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3986, 0.6014, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2526, 0.3791, 0.3683, 0.0000, 0.0000, 0.0000],
        [0.2265, 0.2839, 0.2794, 0.2103, 0.0000, 0.0000],
        [0.1952, 0.2363, 0.2331, 0.1820, 0.1534, 0.0000],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])


# dropout implementation

In [74]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #A
example = torch.ones(6, 6) #B
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [73]:
torch.manual_seed(123)
torch.dropout(example,p=0.5,train=True)

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])

During training, dropout is active. During inference, dropout is off. If we didn't scale, the model would see different magnitude values at train vs test time, breaking predictions.

With scaling, no adjustment is needed at inference — the expected values match.

In [75]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5052, 0.7582, 0.7366, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5677, 0.5587, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4727, 0.0000, 0.3639, 0.0000, 0.0000],
        [0.0000, 0.4183, 0.4097, 0.2839, 0.2178, 0.0000]])


# batch demonstration 


In [76]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) 

torch.Size([2, 6, 3])


In [77]:
batch

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])